In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
titanic_df = pd.read_csv("data/titanic.csv")

In [ ]:
numeric_val = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
titanic_df[numeric_val].info()

In [ ]:
titanic_df.describe()

In [ ]:
#Age에 결측값있어서 중앙값 대체
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].median())

In [ ]:
titanic_df[numeric_val].info()

In [ ]:
#결과가 잘 나올것이라 판단되는 Age, Fare만 이상치 탐지 진행

In [ ]:
#Age
Q1 = titanic_df['Age'].quantile(0.25)
Q3 = titanic_df['Age'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - (1.5 * IQR)
upper = Q3 + (1.5 * IQR)

print(f"Age IQR 범위: {lower:.2f} ~ {upper:.2f}")
print(f"Age IQR 이상치 수: {(titanic_df['Age'] < lower).sum() + (titanic_df['Age'] > upper).sum()}")

In [ ]:
sns.boxplot(x=titanic_df['Age'])
plt.show()

In [ ]:
#Fare
Q1 = titanic_df['Fare'].quantile(0.25)
Q3 = titanic_df['Fare'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - (1.5 * IQR)
upper = Q3 + (1.5 * IQR)

print(f"Fare IQR 범위: {lower:.2f} ~ {upper:.2f}")
print(f"Fare IQR 이상치 수: {(titanic_df['Fare'] < lower).sum() + (titanic_df['Fare'] > upper).sum()}")

In [ ]:
sns.boxplot(x=titanic_df['Fare'])
plt.show()

In [ ]:
def check_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - (1.5 * IQR)
    upper = Q3 + (1.5 * IQR)
    outliers = data[(data[column] < lower) | (data[column] > upper)]
    print(f"{column} IQR 범위: {lower:.2f} ~ {upper:.2f}")
    print(f"Fare IQR 이상치 수: {(data[column] < lower).sum() + (data[column] > upper).sum()}")
    return outliers

In [ ]:
for col in ['Age', 'Fare']:
    outliers = check_outliers_iqr(titanic_df, col)
    sns.boxplot(x=titanic_df[col])
    plt.show()

In [ ]:
#Z-score 방식
from scipy.stats import zscore

titanic_z = titanic_df[['Age', 'Fare']].copy()
z_scores = zscore(titanic_z)
titanic_z = pd.DataFrame(z_scores, columns=['Age', 'Fare'])

In [ ]:
for col in titanic_z.columns:
    count = (np.abs(titanic_z[col]) > 3).sum()
    print(f"{col} z-score 이상치 수: {count}")
    sns.histplot(titanic_z[col], kde=True)
    plt.show()

In [ ]:
#히스토그램상 age는 정규분포에 가깝고, Fare는 왼쪽으로 많이 치우쳐져 있음

In [ ]:
# 상관관계 분석
corr = titanic_df[numeric_val].corr()
corr

In [ ]:
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('corr heatmap')
plt.show()

In [ ]:
"""
상관계수의 판단은 대략 절댓값 기준
0.00 ~ 0.10 : 매우 약함 (무시 가능), 거의 무상관, 관련성 없음
0.10 ~ 0.30 : 약한 상관, 약한 관련성, 데이터 노이즈 수준
0.30 ~ 0.50 : 중간 정도, 유의미한 상관 관계 가능성
0.50 ~ 0.70 : 강한 상관, 명확한 관련성, 인과 가능성 탐색 대상
0.70 ~ 0.90 : 매우 강한 상관, 변수 간 관계가 뚜렷함, 다중공선성 가능성 있음
0.90 ~ 1.00 : 거의 완벽한 상관, 변수 간 중복 가능성, 동일 정보 수준
"""

In [ ]:
# 상관계수 절댓값 기준으로 필터링
threshold = 0.5
strong_corrs = (corr.abs() > threshold) & (corr.abs() < 1.0)

for col in strong_corrs.columns:
    related = strong_corrs.index[strong_corrs[col]].tolist()
    if related:
        print(f"{col} ↔ {related}")

In [ ]:
print(type(corr))